In [21]:
import numpy as np
with open("test", "rb") as file:
    data = file.read()
test = np.frombuffer(data, dtype = "float32")


In [24]:
test.reshape(-1, 3)

array([[ 390.96243  ,    2.4847636,   25.577541 ],
       [  31.978739 , 1003.7829   , 1092.0242   ],
       [ 400.44894  ,    9.536903 ,  153.74039  ],
       ...,
       [ 167.22112  ,  508.1221   ,  346.83237  ],
       [  87.97818  , 1509.6582   , 1387.479    ],
       [ 306.47092  ,  508.57092  ,  407.18906  ]], dtype=float32)

In [10]:
test = test.reshape(-1, 3)
test.shape

ValueError: cannot reshape array of size 262144 into shape (3)

In [22]:
inds = test[:, 0].argsort()[::-1]

In [23]:
test[inds, :]

array([[36269.863   ,    63.411865,   430.2017  ],
       [34819.504   ,   334.08057 ,   235.57753 ],
       [29495.549   ,   177.48776 ,   257.4005  ],
       ...,
       [  149.01945 ,   284.0161  ,   508.43948 ],
       [  116.09711 ,   508.28912 ,   481.55927 ],
       [  108.05166 ,   347.6749  ,   507.60526 ]], dtype=float32)